## 🧠 Arquitectura completa del modelo — `SmallCNN_Res`

Este modelo está formado por **bloques convolucionales residuales** que extraen características visuales a diferentes escalas,  
y una **parte densa final (clasificador)** que convierte esas características en una decisión binaria (*Benign / Malignant*).

---


### 🧩 Arquitectura del modelo

#### 🖼 Entrada  
**Imagen RGB:** `[3 × 224 × 224]`

---

#### 🧱 Bloque 1 — ResidualBlock(3 → 64)
- Conv2d(3, 64, kernel_size=3, padding=1)  
- BatchNorm2d(64)  
- SiLU  
- Conv2d(64, 64, kernel_size=3, padding=1)  
- BatchNorm2d(64)  
- SiLU  
- Dropout2d(0.20)  
- Residual (1×1 conv si cambia canales)  
- SiLU  
- MaxPool2d(2,2)  
📏 **Salida:** `[64 × 112 × 112]`

---

#### 🧱 Bloque 2 — ResidualBlock(64 → 128)
- Conv2d(64, 128, kernel_size=3, padding=1)  
- BatchNorm2d(128)  
- SiLU  
- Conv2d(128, 128, kernel_size=3, padding=1)  
- BatchNorm2d(128)  
- SiLU  
- Dropout2d(0.20)  
- Residual (1×1 conv)  
- SiLU  
- MaxPool2d(2,2)  
📏 **Salida:** `[128 × 56 × 56]`

---

#### 🧱 Bloque 3 — ResidualBlock(128 → 256)
- Conv2d(128, 256, kernel_size=3, padding=1)  
- BatchNorm2d(256)  
- SiLU  
- Conv2d(256, 256, kernel_size=3, padding=1)  
- BatchNorm2d(256)  
- SiLU  
- Dropout2d(0.20)  
- Residual (1×1 conv)  
- SiLU  
- MaxPool2d(2,2)  
📏 **Salida:** `[256 × 28 × 28]`

---

#### 🧱 Bloque 4 — ResidualBlock(256 → 256)
- Conv2d(256, 256, kernel_size=3, padding=1)  
- BatchNorm2d(256)  
- SiLU  
- Conv2d(256, 256, kernel_size=3, padding=1)  
- BatchNorm2d(256)  
- SiLU  
- Dropout2d(0.20)  
- Residual (sin proyección)  
- SiLU  
- MaxPool2d(2,2)  
📏 **Salida:** `[256 × 14 × 14]`

---

#### 🔄 Global Average Pooling
- AdaptiveAvgPool2d((1,1))  
📏 **Salida:** `[256 × 1 × 1]`

---

#### 🧠 Clasificador (Fully Connected)
- Flatten() → `[256]`  
- Linear(256 → 128)  
- SiLU  
- Dropout(0.50)  
- Linear(128 → 2)  

📤 **Salida final:** `[2]`  
*(Probabilidades: Benign / Malignant)*

---

### 💬 Interpretación general

| Etapa | Tipo de capa | Tamaño de salida | Descripción |
|:------|:--------------|:----------------:|:-------------|
| Entrada | Imagen RGB | [3×224×224] | Imagen normalizada de entrada |
| Bloque 1 | Residual conv | [64×112×112] | Detecta bordes, colores y texturas básicas |
| Bloque 2 | Residual conv | [128×56×56] | Aprende formas intermedias y estructuras |
| Bloque 3 | Residual conv | [256×28×28] | Capta regiones más amplias y patrones complejos |
| Bloque 4 | Residual conv | [256×14×14] | Refina las características más profundas |
| GAP | Global Avg Pool | [256×1×1] | Resume cada mapa de activación a un valor medio |
| FC1 | Densa | [128] | Combina características extraídas |
| FC2 | Densa (salida) | [2] | Clasifica en Benigno / Maligno |

---

### ⚙️ Resumen técnico

- 🔹 **Capas convolucionales totales:** 8 (2 por bloque × 4 bloques)  
- 🔹 **Capas residuales:** 4 (cada una con posible proyección 1×1)  
- 🔹 **Capas densas:** 2  
- 🔹 **Funciones de activación:** `SiLU` (suave y estable, ideal para evitar saturación)  
- 🔹 **Regularización:** `Dropout(0.20)` en convs y `Dropout(0.50)` en FC  
- 🔹 **Tamaño de entrada:** 224×224 píxeles  
- 🔹 **Tamaño de salida:** 2 neuronas (clasificación binaria)  
- 🔹 **Parámetros entrenables aprox.:** entre **3,5 y 4 millones**

---

### 🧩 Resumen conceptual

Este modelo combina:
- **Bloques residuales profundos**, que facilitan el flujo del gradiente y reducen el sobreajuste.  
- **Capas convolucionales dobles**, que permiten extraer rasgos jerárquicos complejos.  
- **Pooling progresivo**, que reduce la dimensionalidad manteniendo la información esencial.  
- **Un clasificador totalmente conectado**, que interpreta las representaciones y decide si la lesión es benigna o maligna.

---

### ✅ En resumen

Tu `SmallCNN_Res` es una CNN **profunda, eficiente y con buena capacidad de generalización**,  
capaz de analizar patrones complejos en imágenes dermatoscópicas y **distinguir entre melanomas malignos y lesiones benignas**.

---


## 🧩 Celda 1 – Configuración inicial

Define todas las **importaciones, rutas y parámetros globales** del proyecto:
- Establece las carpetas de trabajo (`train`, `val`, `test`).
- Fija las **semillas aleatorias** para asegurar reproducibilidad.
- Configura el **dispositivo de entrenamiento** (en este caso, CPU).
- Define hiperparámetros como `BATCH`, `EPOCHS`, `LR` y el tamaño de imagen.

> 💡 Esta celda no genera resultados visibles, pero es clave para mantener consistencia y evitar comportamientos aleatorios en el modelo.


In [5]:
# === Celda 1: Imports, rutas, device y parámetros (CPU) ===
from pathlib import Path
import os, time, random
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, WeightedRandomSampler
from torchvision import datasets, transforms

# ---- Rutas (ajústalas si hace falta) ----
DATA_DIR = Path(r"D:/proyectos/Caso_aprendizaje-Melanomas/data/raw")
TRAIN_DIR = DATA_DIR / "train"
VAL_DIR   = DATA_DIR / "val"
TEST_DIR  = DATA_DIR / "test"

for p in (TRAIN_DIR, VAL_DIR, TEST_DIR):
    print(p, "exists:", p.exists())

# ---- Semillas fijas ----
SEED = 42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)

# ---- Device (solo CPU) + optimizaciones para CPU ----
device = torch.device("cpu")
print("Device for training:", device)

# Usa la mayoría de núcleos para acelerar en CPU
num_threads = max(1, (os.cpu_count() or 4) - 1)
torch.set_num_threads(num_threads)
try:
    torch.set_float32_matmul_precision("high")  # mejora BLAS en CPU modernas
except Exception:
    pass
print(f"CPU threads: {num_threads}")

# Desactiva flags de CUDA (no aplican en CPU)
torch.backends.cudnn.deterministic = False
torch.backends.cudnn.benchmark = False

# ---- Parámetros de datos/entreno (pensados para CPU) ----
CLASSES = 2
ROWS = COLS = 224        # puedes subir a 256 si te cabe en RAM y va fluido
BATCH = 16               # en CPU suele aguantar más batch que tu GPU de 4GB
EPOCHS = 20              # más épocas para apretar el fine-tune
LR = 3e-4
WEIGHT_DECAY = 1e-4
NUM_WORKERS = 4          # en CPU paraleliza carga de datos (prueba 4–8 según equipo)
PIN_MEMORY = False       # en CPU no hace falta
USE_AMP = False          # AMP solo aporta en GPU

# ---- Normalización (ImageNet) ----
IMAGENET_MEAN = (0.485, 0.456, 0.406)
IMAGENET_STD  = (0.229, 0.224, 0.225)


D:\proyectos\Caso_aprendizaje-Melanomas\data\raw\train exists: True
D:\proyectos\Caso_aprendizaje-Melanomas\data\raw\val exists: True
D:\proyectos\Caso_aprendizaje-Melanomas\data\raw\test exists: True
Device for training: cpu
CPU threads: 7


## 🧠 Celda 2 – Transformaciones y carga de datos

Aplica un **tratamiento y aumento de datos (data augmentation)** moderado para el conjunto de entrenamiento:
- Rotaciones, volteos, recortes aleatorios, ajustes de brillo y contraste.
- Normalización según los valores de *ImageNet*.
- Balancea las clases usando un **WeightedRandomSampler**, para evitar que la clase "Benign" domine.

> 📊 El objetivo es que el modelo vea imágenes variadas de las lesiones y aprenda a reconocer patrones robustos, evitando el sobreajuste a casos muy concretos.


In [6]:
# === Celda 2: Features + Dataset con features + Transforms + Pesos + Caché + DataLoaders ===
# Requisitos previos en Celda 1:
# DATA_DIR, TRAIN_DIR, VAL_DIR, CLASSES, ROWS, COLS, BATCH, NUM_WORKERS, PIN_MEMORY, IMAGENET_MEAN, IMAGENET_STD

# ------------------ Imports base ------------------
import os, json, time
import numpy as np
import torch
from torch.utils.data import DataLoader, WeightedRandomSampler
from torchvision import transforms, datasets
from torchvision.datasets import ImageFolder
from torchvision.transforms import functional as TF
from tqdm.auto import tqdm

# --- libs para features ---
import cv2
from skimage import morphology, measure, color, filters
from sklearn.cluster import KMeans


# ======================================================================
# 2.5  EXTRACCIÓN DE FEATURES (segmentación + morfología + color)
# ======================================================================
def largest_component(mask: np.ndarray) -> np.ndarray:
    labeled = measure.label(mask, connectivity=2)
    if labeled.max() == 0:
        return mask
    counts = np.bincount(labeled.ravel())
    counts[0] = 0
    keep = counts.argmax()
    return (labeled == keep).astype(np.uint8)

def lesion_mask_binarize(img_bgr: np.ndarray) -> np.ndarray:
    """Segmentación rápida basada en V (HSV) + Otsu + morfología."""
    hsv = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2HSV)
    v   = hsv[...,2]
    v_blur = cv2.GaussianBlur(v, (5,5), 0)
    thr = filters.threshold_otsu(v_blur)
    mask = (v_blur < thr).astype(np.uint8)  # lesión suele ser más oscura

    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE,
                            cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7,7)), iterations=2)
    mask = morphology.remove_small_objects(mask.astype(bool), min_size=200).astype(np.uint8)
    mask = morphology.remove_small_holes(mask.astype(bool), area_threshold=200).astype(np.uint8)
    mask = largest_component(mask)
    return mask

def symmetry_scores(mask: np.ndarray) -> tuple[float,float]:
    """Asimetría izquierda-derecha y arriba-abajo (IoU; 1=simétrico)."""
    h, w = mask.shape
    left  = mask[:, :w//2]
    right = np.fliplr(mask[:, w - w//2:])
    top   = mask[:h//2, :]
    bot   = np.flipud(mask[h - h//2:, :])

    def iou(a,b):
        inter = np.logical_and(a,b).sum()
        union = np.logical_or(a,b).sum()
        return inter/union if union>0 else 0.0

    return iou(left, right), iou(top, bot)

def color_features(img_bgr: np.ndarray, mask: np.ndarray, k_colors: int = 3) -> dict:
    """Medias/desv de HSV en la máscara + diversidad de color (KMeans en S,V)."""
    hsv = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2HSV)
    m = mask.astype(bool)
    if m.sum() == 0:
        return {k:0.0 for k in ["h_mean","h_std","s_mean","s_std","v_mean","v_std","color_entropy","kspread"]}

    H = hsv[...,0][m].astype(np.float32)         # 0..180 en OpenCV
    S = hsv[...,1][m].astype(np.float32)/255.0
    V = hsv[...,2][m].astype(np.float32)/255.0

    feats = {
        "h_mean": float(H.mean()), "h_std": float(H.std()),
        "s_mean": float(S.mean()), "s_std": float(S.std()),
        "v_mean": float(V.mean()), "v_std": float(V.std()),
    }

    sv = np.stack([S, V], axis=1)
    k = min(k_colors, len(sv))
    if k >= 2:
        km = KMeans(n_clusters=k, n_init=5, random_state=42)
        labels = km.fit_predict(sv)
        counts = np.bincount(labels, minlength=k).astype(np.float32)
        p = counts / counts.sum()
        entropy = -np.sum(p * np.log(p + 1e-12))
        centers = km.cluster_centers_
        kspread = float(np.linalg.norm(centers.max(0) - centers.min(0)))
        feats["color_entropy"] = float(entropy)
        feats["kspread"] = kspread
    else:
        feats["color_entropy"] = 0.0
        feats["kspread"] = 0.0

    return feats

def shape_features(mask: np.ndarray) -> dict:
    """Área relativa, perímetro relativo, circularidad, irregularidad, excentricidad."""
    m = mask.astype(bool)
    area = m.sum()
    if area == 0:
        return {k:0.0 for k in [
            "area_rel","perim_rel","circularity","irregularity","eccentricity","sym_lr","sym_tb"
        ]}
    h, w = m.shape

    # ✅ skimage ≥0.20 usa 'neighborhood'; algunas versiones antiguas aceptaban 'neighbourhood'
    try:
        perim = measure.perimeter(m, neighborhood=8)
    except TypeError:
        # Fallback para versiones antiguas (por si acaso)
        perim = measure.perimeter(m, neighbourhood=8)

    area_rel  = area / (h*w)
    perim_rel = perim / (h+w)
    circularity = (4*np.pi*area) / (perim**2 + 1e-12)
    irregularity = (perim**2) / (4*np.pi*area + 1e-12)

    props = measure.regionprops(m.astype(np.uint8))[0]
    eccentricity = float(props.eccentricity)

    sym_lr, sym_tb = symmetry_scores(mask)
    return {
        "area_rel": float(area_rel),
        "perim_rel": float(perim_rel),
        "circularity": float(circularity),
        "irregularity": float(irregularity),
        "eccentricity": float(eccentricity),
        "sym_lr": float(sym_lr),
        "sym_tb": float(sym_tb),
    }


def extract_features_from_path(img_path: str) -> tuple[np.ndarray, dict, np.ndarray]:
    """Devuelve (img_bgr, dict_features, mask)."""
    img_bgr = cv2.imread(str(img_path), cv2.IMREAD_COLOR)
    if img_bgr is None:
        raise FileNotFoundError(img_path)
    mask = lesion_mask_binarize(img_bgr)
    f_shape = shape_features(mask)
    f_color = color_features(img_bgr, mask, k_colors=3)
    feats = {**f_shape, **f_color}
    return img_bgr, feats, mask


# ======================================================================
# 2.6  DATASET que añade vector de features al sample
# ======================================================================
FEATURE_KEYS = [
    "area_rel","perim_rel","circularity","irregularity","eccentricity",
    "sym_lr","sym_tb","h_mean","h_std","s_mean","s_std","v_mean","v_std",
    "color_entropy","kspread"
]

class ImageFolderWithFeatures(ImageFolder):
    def __init__(self, root, transform=None, cache=True):
        super().__init__(root, transform=transform)
        self.cache = cache
        self._cache = {}

    def _get_feats(self, path):
        if self.cache and path in self._cache:
            return self._cache[path]
        _, feats, _ = extract_features_from_path(path)
        x = np.array([feats[k] for k in FEATURE_KEYS], dtype=np.float32)
        if self.cache:
            self._cache[path] = x
        return x

    def __getitem__(self, idx):
        path, target = self.samples[idx]
        sample = self.loader(path)  # PIL
        img = self.transform(sample) if self.transform is not None else TF.to_tensor(sample)
        feats = torch.from_numpy(self._get_feats(path))  # [F]
        return img, target, feats


# ======================================================================
# 2.0  TRANSFORMS + pesos por clase (antes de DataLoaders)
# ======================================================================
train_tfms = transforms.Compose([
    transforms.RandomResizedCrop((ROWS, COLS), scale=(0.70, 1.00), ratio=(0.90, 1.10)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(15, fill=(128, 128, 128)),
    transforms.ColorJitter(brightness=0.20, contrast=0.20, saturation=0.10, hue=0.02),
    transforms.ToTensor(),
    transforms.Normalize(IMAGENET_MEAN, IMAGENET_STD),
    transforms.RandomErasing(p=0.25, scale=(0.02, 0.08), ratio=(0.3, 3.3)),
])

val_tfms = transforms.Compose([
    transforms.Resize((ROWS, COLS)),
    transforms.ToTensor(),
    transforms.Normalize(IMAGENET_MEAN, IMAGENET_STD),
])

# Descubrir clases/pesos
_tmp_ds = datasets.ImageFolder(TRAIN_DIR)
print("Clases detectadas:", _tmp_ds.classes)
assert len(_tmp_ds.classes) == CLASSES, f"Esperaba {CLASSES} clases, encontré {len(_tmp_ds.classes)}."
targets_for_weights = np.array([y for _, y in _tmp_ds.samples])
CLASS_COUNTS_NP  = np.bincount(targets_for_weights, minlength=CLASSES)
CLASS_WEIGHTS_NP = 1.0 / (CLASS_COUNTS_NP + 1e-6)
CLASSES_LIST     = _tmp_ds.classes
print("Class counts:", CLASS_COUNTS_NP, "-> weights:", np.round(CLASS_WEIGHTS_NP, 4))
del _tmp_ds

# ======================================================================
# 2.3  Crear DATASETS con features
# ======================================================================
train_ds = ImageFolderWithFeatures(TRAIN_DIR, transform=train_tfms, cache=True)
val_ds   = ImageFolderWithFeatures(VAL_DIR,   transform=val_tfms,   cache=True)
print("Clases (dataset real):", train_ds.classes)

# ======================================================================
# 2.7  CACHÉ de features (cargar si existe; si no, precalcular y guardar)
# ======================================================================
CACHE_DIR = DATA_DIR / "_feats_cache"
os.makedirs(CACHE_DIR, exist_ok=True)

def precache_features(ds):
    for path, _ in tqdm(ds.samples, total=len(ds.samples), desc=f"Precacheando {os.path.basename(ds.root)}"):
        _ = ds._get_feats(path)

def save_cache(ds, out_json):
    serial = {p: ds._cache[p].tolist() for p,_ in ds.samples if p in ds._cache}
    with open(out_json, "w", encoding="utf-8") as f:
        json.dump(serial, f)

def load_cache(ds, in_json):
    if not os.path.exists(in_json):
        return False
    with open(in_json, "r", encoding="utf-8") as f:
        data = json.load(f)
    ds._cache.update({k: np.array(v, dtype=np.float32) for k, v in data.items()})
    print(f"Cache cargada desde: {in_json} ({len(data)} entradas)")
    return True

loaded_train = load_cache(train_ds, str(CACHE_DIR / "train_feats.json"))
loaded_val   = load_cache(val_ds,   str(CACHE_DIR / "val_feats.json"))

if not loaded_train:
    print("⚙️ Precaching TRAIN (primera vez)...")
    precache_features(train_ds)
    save_cache(train_ds, str(CACHE_DIR / "train_feats.json"))

if not loaded_val:
    print("⚙️ Precaching VAL (primera vez)...")
    precache_features(val_ds)
    save_cache(val_ds, str(CACHE_DIR / "val_feats.json"))

print("✅ Features listas en memoria.")

# ======================================================================
# 2.8  DATALOADERS balanceados (con collate que incluye features)
# ======================================================================
targets = np.array([y for _, y in train_ds.samples])
sample_weights = CLASS_WEIGHTS_NP[targets]
sampler = WeightedRandomSampler(weights=sample_weights,
                                num_samples=len(sample_weights),
                                replacement=True)

def collate_with_feats(batch):
    imgs = torch.stack([b[0] for b in batch], dim=0)
    ys   = torch.tensor([b[1] for b in batch], dtype=torch.long)
    fs   = torch.stack([b[2] for b in batch], dim=0)
    return imgs, ys, fs

persistent = True if NUM_WORKERS and NUM_WORKERS > 0 else False

train_dl = DataLoader(train_ds,
                      batch_size=BATCH,
                      sampler=sampler,
                      num_workers=NUM_WORKERS,
                      pin_memory=PIN_MEMORY,
                      persistent_workers=persistent,
                      collate_fn=collate_with_feats)

val_dl = DataLoader(val_ds,
                    batch_size=BATCH,
                    shuffle=False,
                    num_workers=NUM_WORKERS,
                    pin_memory=PIN_MEMORY,
                    persistent_workers=persistent,
                    collate_fn=collate_with_feats)

print("✅ DataLoaders listos.")
print(f"   Train: {len(train_ds)} imgs | Val: {len(val_ds)} imgs")


Clases detectadas: ['Benign', 'Malignant']
Class counts: [5346 4752] -> weights: [0.0002 0.0002]
Clases (dataset real): ['Benign', 'Malignant']
⚙️ Precaching TRAIN (primera vez)...


Precacheando train: 100%|██████████| 10098/10098 [09:45<00:00, 17.25it/s]


⚙️ Precaching VAL (primera vez)...


Precacheando val: 100%|██████████| 1781/1781 [01:45<00:00, 16.90it/s]


✅ Features listas en memoria.
✅ DataLoaders listos.
   Train: 10098 imgs | Val: 1781 imgs


## 🔬 Celda 3 – Definición del modelo convolucional mejorado

Define la arquitectura `SmallCNN_Res_Fusion`, basada en **bloques residuales**:
- 4 bloques de convoluciones con conexiones *skip* para estabilidad.
- Función de activación *SiLU* (más suave que ReLU).
- Promedio global (GAP) + capa totalmente conectada con fusión de features.

> 💪 Este modelo es más profundo y estable que la versión simple.  
> Gracias a las conexiones residuales, se entrena mejor incluso con conjuntos de datos limitados y capta texturas y estructuras más complejas de la piel.


In [7]:
# === Celda 3: Modelo CNN + FUSIÓN de features (doble conv + residual) ===
import torch
import torch.nn as nn
from typing import Optional

# Nº de features adicionales (definido por tu Celda 2.6)
N_FEATS = len(FEATURE_KEYS)  # p.ej., 15

class DoubleConv(nn.Module):
    """Conv3x3 -> BN -> SiLU -> Conv3x3 -> BN -> SiLU -> Dropout2d"""
    def __init__(self, in_ch, out_ch, drop=0.20):
        super().__init__()
        self.block = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, 3, 1, 1, bias=False),
            nn.BatchNorm2d(out_ch),
            nn.SiLU(inplace=True),

            nn.Conv2d(out_ch, out_ch, 3, 1, 1, bias=False),
            nn.BatchNorm2d(out_ch),
            nn.SiLU(inplace=True),

            nn.Dropout2d(drop),
        )
    def forward(self, x):
        return self.block(x)

class ResidualBlock(nn.Module):
    """
    x ──► DoubleConv ──► + (proyección 1x1 si cambia canal) ──► SiLU ──► MaxPool(2)
    """
    def __init__(self, in_ch, out_ch, drop=0.20):
        super().__init__()
        self.double = DoubleConv(in_ch, out_ch, drop=drop)
        self.proj: Optional[nn.Module] = None
        if in_ch != out_ch:
            self.proj = nn.Sequential(
                nn.Conv2d(in_ch, out_ch, 1, bias=False),
                nn.BatchNorm2d(out_ch)
            )
        self.act  = nn.SiLU(inplace=True)
        self.pool = nn.MaxPool2d(2, 2)

    def forward(self, x):
        y = self.double(x)
        skip = x if self.proj is None else self.proj(x)
        y = self.act(y + skip)
        y = self.pool(y)
        return y

class SmallCNN_Res_Fusion(nn.Module):
    """
    4 bloques residuales (8 convs): 224→112→56→28→14
    GAP -> concat([feat_map, features_extras]) -> MLP -> logits
    """
    def __init__(self, in_ch=3, num_classes=2, drop_conv=0.20, drop_fc=0.50, n_feats=N_FEATS):
        super().__init__()
        self.block1 = ResidualBlock(in_ch,   64,  drop=drop_conv)
        self.block2 = ResidualBlock(64,      128, drop=drop_conv)
        self.block3 = ResidualBlock(128,     256, drop=drop_conv)
        self.block4 = ResidualBlock(256,     256, drop=drop_conv)

        self.gap = nn.AdaptiveAvgPool2d((1,1))  # [B,256,1,1]
        self.flatten = nn.Flatten()             # -> [B,256]
        self.n_feats = n_feats

        # Cabeza de fusión: concatena [256] con [n_feats]
        self.head = nn.Sequential(
            nn.Linear(256 + n_feats, 128),
            nn.SiLU(inplace=True),
            nn.Dropout(drop_fc),
            nn.Linear(128, num_classes),
        )

        # Inicialización Kaiming para estabilidad
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode="fan_out", nonlinearity="relu")
            elif isinstance(m, nn.Linear):
                nn.init.kaiming_normal_(m.weight, mode="fan_in", nonlinearity="relu")
                if m.bias is not None:
                    nn.init.zeros_(m.bias)

    def forward(self, x, feats):
        # x: [B,3,H,W], feats: [B, n_feats]
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.block4(x)
        x = self.gap(x)
        x = self.flatten(x)                 # [B,256]
        # Asegura tipo/shape de feats
        if feats.dim() == 1:
            feats = feats.unsqueeze(0)
        feats = feats.to(x.dtype)
        z = torch.cat([x, feats], dim=1)    # [B, 256+n_feats]
        out = self.head(z)                  # [B,num_classes]
        return out

# Instanciación
model = SmallCNN_Res_Fusion(in_ch=3, num_classes=CLASSES,
                            drop_conv=0.20, drop_fc=0.50, n_feats=N_FEATS).to(device)
params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Parámetros entrenables (fusión): {params:,}")


Parámetros entrenables (fusión): 2,404,098


## ⚙️ Celda 4 – Definición de la función de pérdida y optimizador

- Usa **CrossEntropyLoss** con pesos por clase para corregir el desequilibrio entre "Benign" y "Malignant".
- Emplea el optimizador **AdamW**, que combina Adam con regularización L2 (controla el sobreajuste).
- Define la métrica de precisión (`accuracy_from_logits`).

> ⚖️ Esta celda ajusta cómo el modelo aprende (descenso del gradiente) y penaliza los errores de forma equilibrada entre las dos clases.


In [8]:
# === Celda 4: Loss, optimizador y métrica (CPU) ===
import torch.nn as nn
import torch

# Toma los pesos de clase desde la Celda 2 (fallback si cambiaste el nombre)
_weights_np = globals().get("CLASS_WEIGHTS_NP", globals().get("class_weights", None))
assert _weights_np is not None, "No encuentro CLASS_WEIGHTS_NP ni class_weights (revisa Celda 2)."

w = torch.tensor(_weights_np, dtype=torch.float32, device=device)

# Pérdida ponderada + label smoothing suave
criterion = nn.CrossEntropyLoss(weight=w, label_smoothing=0.05)

# Optimizador con buen decay (mejor que Adam para fine-tune)
trainable_params = filter(lambda p: p.requires_grad, model.parameters())
optimizer = torch.optim.AdamW(trainable_params, lr=LR, weight_decay=WEIGHT_DECAY)

# Métrica principal: accuracy
def accuracy_from_logits(logits, targets):
    preds = logits.argmax(dim=1)
    return (preds == targets).float().mean().item()

print("Optimizer: AdamW | LR =", LR, "| weight_decay =", WEIGHT_DECAY)


Optimizer: AdamW | LR = 0.0003 | weight_decay = 0.0001


## 🚀 Celda 5 – Entrenamiento y validación

Ejecuta el bucle de entrenamiento:
1. **Entrena** el modelo sobre `train_dl` durante varias épocas.
2. **Evalúa** sobre `val_dl` en cada época.
3. Usa **early stopping** si la pérdida de validación deja de mejorar.
4. Guarda el mejor modelo (`.pth`) basado en la pérdida mínima.

> 📈 Aquí se observa si el modelo realmente aprende:  
> - La *pérdida de entrenamiento* debe disminuir progresivamente.  
> - La *pérdida de validación* no debe aumentar mucho (evita overfitting).  
> - Las *precisiones* de train y val deben estar próximas si el modelo generaliza bien.


In [ ]:
# === Celda 5: Entreno/validación (CPU, sin AMP) — compatible con features ===

def _unpack_batch(batch):
    # batch puede ser (x,y,f) o (x,y)
    if isinstance(batch, (list, tuple)) and len(batch) == 3:
        x, y, f = batch
    else:
        x, y = batch
        f = None
    return x, y, f

def train_one_epoch(model, loader, optimizer, criterion, clip_grad_norm=1.0, log_every=20):
    """
    Entrena una época completa y muestra progreso por batches.
    log_every: muestra info cada N batches.
    """
    model.train()
    total_loss = total_acc = n = 0
    n_batches = len(loader)

    start_time = time.time()

    for i, batch in enumerate(loader):
        x, y, f = _unpack_batch(batch)
        x = x.to(device); y = y.to(device)
        if f is not None: f = f.to(device)

        optimizer.zero_grad(set_to_none=True)
        logits = model(x, f) if f is not None else model(x)
        loss = criterion(logits, y)
        loss.backward()

        # (opcional) clipping para estabilidad en CPU
        if clip_grad_norm and clip_grad_norm > 0:
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=clip_grad_norm)

        optimizer.step()

        total_loss += loss.item()
        total_acc  += accuracy_from_logits(logits, y)
        n += 1

        # 🔹 Mostrar progreso cada log_every batches
        if (i + 1) % log_every == 0 or (i + 1) == n_batches:
            pct = 100 * (i + 1) / n_batches
            avg_loss = total_loss / n
            avg_acc  = total_acc / n
            elapsed = time.time() - start_time
            print(f"[Batch {i+1:4d}/{n_batches}] {pct:5.1f}% | Loss={avg_loss:.4f} | "
                  f"Acc={avg_acc:.4f} | Tiempo={elapsed:.1f}s")

    epoch_loss = total_loss / max(n, 1)
    epoch_acc  = total_acc / max(n, 1)
    print(f"✅ Época completada: loss={epoch_loss:.4f}, acc={epoch_acc:.4f}, "
          f"duración total={time.time()-start_time:.1f}s\n")

    return epoch_loss, epoch_acc


@torch.no_grad()
def validate(model, loader, criterion):
    model.eval()
    total_loss = total_acc = n = 0
    for batch in loader:
        x, y, f = _unpack_batch(batch)
        x = x.to(device); y = y.to(device)
        if f is not None: f = f.to(device)

        logits = model(x, f) if f is not None else model(x)
        loss = criterion(logits, y)

        total_loss += loss.item()
        total_acc  += accuracy_from_logits(logits, y)
        n += 1
    return total_loss / max(n, 1), total_acc / max(n, 1)

history = {"train_loss":[], "train_acc":[], "val_loss":[], "val_acc":[]}
BEST_PATH = "smallcnn_res_fusion_cpu_best.pth"  # nombre coherente con el modelo fusionado

# === Scheduler + Early Stopping (usa el optimizer de Celda 4) ===
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='min', factor=0.5, patience=3, verbose=True
)

EARLY_PATIENCE = 6
no_improve = 0
best_val = float('inf')

for epoch in range(1, EPOCHS+1):
    t0 = time.time()

    tr_loss, tr_acc = train_one_epoch(model, train_dl, optimizer, criterion, clip_grad_norm=1.0)
    va_loss, va_acc = validate(model, val_dl, criterion)

    # guarda historial para las curvas
    history["train_loss"].append(tr_loss); history["train_acc"].append(tr_acc)
    history["val_loss"].append(va_loss);   history["val_acc"].append(va_acc)

    # step del scheduler en función de val_loss
    scheduler.step(va_loss)

    # guarda mejor checkpoint por val_loss
    if va_loss < best_val:
        best_val = va_loss
        torch.save({
            "model": model.state_dict(),
            "classes": CLASSES_LIST if 'CLASSES_LIST' in globals() else None,
            "img_size": (ROWS, COLS),
            "feature_keys": FEATURE_KEYS  # para asegurar compatibilidad al cargar
        }, BEST_PATH)
        no_improve = 0
    else:
        no_improve += 1
        if no_improve >= EARLY_PATIENCE:
            print("Early stopping 🚦")
            break

    dt = time.time() - t0
    cur_lr = optimizer.param_groups[0]["lr"]
    print(f"[{epoch:02d}] train_loss={tr_loss:.4f} acc={tr_acc:.4f} | "
          f"val_loss={va_loss:.4f} acc={va_acc:.4f} | lr={cur_lr:.2e} | {dt:.1f}s")


d:\proyectos\Caso_aprendizaje-Melanomas\.venv\lib\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


## 🧮 Celda 6 – Búsqueda del umbral óptimo

Explora diferentes **umbrales de decisión** para clasificar una imagen como “Malignant”.
- Calcula F1-score, Recall y Precision para cada umbral.
- Escoge el valor que **maximiza el F1-score**, equilibrio entre precisión y sensibilidad.

> 🔍 Este paso mejora la clasificación en problemas médicos, donde el objetivo es reducir **falsos negativos** (no pasar por alto melanomas malignos).


In [ ]:
# === Celda 6: Buscar UMBRAL ÓPTIMO en validación (compatible con features) ===
import numpy as np
from sklearn.metrics import f1_score, recall_score, precision_score
import matplotlib.pyplot as plt
import torch

def _unpack_batch_for_probs(batch):
    # batch puede ser (x,y,f) o (x,y)
    if isinstance(batch, (list, tuple)) and len(batch) == 3:
        x, y, f = batch
    else:
        x, y = batch
        f = None
    return x, y, f

def collect_probs_labels(model, loader, malignant_idx=1):
    """Devuelve prob(Malignant) y etiquetas verdaderas desde un loader."""
    model.eval()
    probs, labels = [], []
    with torch.no_grad():
        for batch in loader:
            x, y, f = _unpack_batch_for_probs(batch)
            x = x.to(device)
            if f is not None:
                f = f.to(device)
                logits = model(x, f)
            else:
                logits = model(x)
            p = torch.softmax(logits, dim=1)[:, malignant_idx].cpu().numpy()
            probs.append(p); labels.append(y.numpy())
    return np.concatenate(probs), np.concatenate(labels)

# 1) Probabilidades en validación para la clase "Malignant" (índice 1)
val_probs, val_y = collect_probs_labels(model, val_dl, malignant_idx=1)

# 2) Barrido de umbrales
ths = np.linspace(0.05, 0.95, 37)
f1s, recs, precs = [], [], []
best_f1, best_t_f1 = -1.0, 0.5
best_rec, best_t_rec = -1.0, 0.5

for t in ths:
    preds = (val_probs >= t).astype(int)
    f1  = f1_score(val_y, preds, pos_label=1, zero_division=0)
    rec = recall_score(val_y, preds, pos_label=1, zero_division=0)
    pre = precision_score(val_y, preds, pos_label=1, zero_division=0)
    f1s.append(f1); recs.append(rec); precs.append(pre)
    if f1 > best_f1: best_f1, best_t_f1 = f1, t
    if rec > best_rec: best_rec, best_t_rec = rec, t

print(f"▪ Umbral óptimo por F1: {best_t_f1:.2f} | F1={best_f1:.3f}")
print(f"▪ Umbral que maximiza Recall: {best_t_rec:.2f} | Recall={best_rec:.3f}")

# 3) Elige el criterio principal
OPT_THRESH = best_t_f1
print(f"OPT_THRESH = {OPT_THRESH:.2f} (usado por defecto)")

# 4) (Opcional) Visualiza F1/Recall/Precision vs Umbral
plt.figure(figsize=(7,4.5))
plt.plot(ths, f1s,  label="F1")
plt.plot(ths, recs, label="Recall")
plt.plot(ths, precs,label="Precision")
plt.axvline(OPT_THRESH, ls="--")
plt.xlabel("Umbral"); plt.ylabel("Métrica")
plt.title("Métricas en Validación vs Umbral")
plt.grid(True, ls="--", alpha=0.5); plt.legend()
plt.tight_layout(); plt.show()


## 🧾 Celda 7 – Evaluación en el conjunto de test

Evalúa el rendimiento del modelo final en las imágenes **nunca vistas**:
- Calcula la precisión total y por clase.  
- Aplica el umbral óptimo obtenido en validación.  

> 🧪 Aquí se obtiene la medida real de rendimiento del modelo: cómo se comportaría con casos nuevos fuera del entrenamiento.
> Si los resultados de *test* son similares a *val*, el modelo **generaliza correctamente**.


In [ ]:
# === Celda 7: Evaluación en test (CPU, compatible con features) ===
from collections import Counter
import torch
from torch.utils.data import DataLoader

# 1) Cargar mejor checkpoint
ckpt = torch.load(BEST_PATH, map_location=device)
model.load_state_dict(ckpt["model"])
model.eval()

# 2) Dataset/Dataloader de test (usa tfms de validación y la clase con features)
test_ds = ImageFolderWithFeatures(TEST_DIR, transform=val_tfms, cache=True)
persistent = True if NUM_WORKERS and NUM_WORKERS > 0 else False

# collate igual que en train_dl/val_dl
def collate_with_feats(batch):
    imgs = torch.stack([b[0] for b in batch], dim=0)
    ys   = torch.tensor([b[1] for b in batch], dtype=torch.long)
    fs   = torch.stack([b[2] for b in batch], dim=0)
    return imgs, ys, fs

test_dl = DataLoader(test_ds,
                     batch_size=BATCH,
                     shuffle=False,
                     num_workers=NUM_WORKERS,
                     pin_memory=PIN_MEMORY,
                     persistent_workers=persistent,
                     collate_fn=collate_with_feats)

# --- Función auxiliar de desempaquetado ---
def _unpack_batch(batch):
    if isinstance(batch, (list, tuple)) and len(batch) == 3:
        x, y, f = batch
    else:
        x, y = batch
        f = None
    return x, y, f

# --- Evaluación estándar (argmax, sin umbral) ---
@torch.no_grad()
def evaluate_loader_argmax(model, loader):
    model.eval()
    total, correct = 0, 0
    per_class = Counter(); per_class_correct = Counter()

    for batch in loader:
        x, y, f = _unpack_batch(batch)
        x = x.to(device); y = y.to(device)
        if f is not None: f = f.to(device)

        logits = model(x, f) if f is not None else model(x)
        preds = logits.argmax(dim=1)
        correct += (preds == y).sum().item()
        total   += y.numel()

        for yi, pi in zip(y.tolist(), preds.tolist()):
            per_class[yi] += 1
            if yi == pi:
                per_class_correct[yi] += 1

    acc = correct / total if total else 0.0
    return acc, per_class, per_class_correct

# --- Evaluación con umbral óptimo (p(Malignant) >= thr) ---
@torch.no_grad()
def evaluate_loader_with_threshold(model, loader, malignant_idx=1, thr=0.5):
    model.eval()
    total, correct = 0, 0
    for batch in loader:
        x, y, f = _unpack_batch(batch)
        x = x.to(device); y = y.to(device)
        if f is not None: f = f.to(device)

        logits = model(x, f) if f is not None else model(x)
        probs = torch.softmax(logits, dim=1)[:, malignant_idx]
        preds = (probs >= thr).long()
        correct += (preds == y).sum().item()
        total   += y.numel()
    return correct / total if total else 0.0

# 3) Evaluación y reporte por clase (argmax)
test_acc, counts, rights = evaluate_loader_argmax(model, test_dl)
print(f"Test accuracy (argmax): {test_acc:.4f}")
idx2class = {i: c for i, c in enumerate(test_ds.classes)}
for i in range(len(test_ds.classes)):
    n = counts[i]; ok = rights[i]; name = idx2class[i]
    if n > 0:
        print(f"  {name:>10s}: {ok}/{n} ({ok/n:.3f})")
    else:
        print(f"  {name:>10s}: 0/0 (N/A)")

# 4) (opcional) Evaluación con umbral óptimo
if 'OPT_THRESH' in globals():
    acc_thr = evaluate_loader_with_threshold(model, test_dl,
                                             malignant_idx=1, thr=OPT_THRESH)
    print(f"\nTest accuracy (umbral óptimo={OPT_THRESH:.2f}): {acc_thr:.4f}")


## 📋 Interpretación del reporte con umbral óptimo

Esta celda evalúa el modelo final usando el **umbral de decisión ajustado (`OPT_THRESH`)**, obtenido en la celda anterior para maximizar el equilibrio entre *precisión* y *recall*.

---

### 🧩 Qué hace el código
1. **`collect_probs_labels`**: calcula las probabilidades de que cada imagen pertenezca a la clase *Malignant*.
2. **`(test_probs >= OPT_THRESH)`**: convierte esas probabilidades en predicciones binarias (`0 = Benign`, `1 = Malignant`).
3. **`classification_report`**: genera métricas detalladas para cada clase.
4. **`confusion_matrix`**: muestra cuántas predicciones fueron correctas o incorrectas en cada categoría.

---

### 📊 Cómo interpretar el reporte

El `classification_report` devuelve cuatro métricas principales para cada clase:

| Métrica | Significado | Interpretación |
|:---------|:-------------|:----------------|
| **Precision** | De todos los casos que el modelo predijo como *Malignant*, cuántos lo eran realmente. | Alta precisión = pocos falsos positivos. |
| **Recall (Sensibilidad)** | De todos los *Malignant* reales, cuántos detectó el modelo. | Alta sensibilidad = pocos falsos negativos. |
| **F1-score** | Media armónica entre precisión y recall. | Resume el equilibrio entre ambas métricas. |
| **Support** | Número real de ejemplos de esa clase en el conjunto de test. | Muestra el tamaño de la muestra. |

Además, el bloque final del reporte muestra:
- **Accuracy global**: porcentaje total de aciertos del modelo.  
- **Macro avg**: media simple entre ambas clases (sin ponderar).  
- **Weighted avg**: media ponderada por la frecuencia de cada clase (más realista si hay desequilibrio).

---

### 🔲 Cómo leer la matriz de confusión

La matriz se organiza así:

|                 | Predicho Benign | Predicho Malignant |
|:----------------|:----------------|:-------------------|
| **Real Benign** | Verdaderos negativos (TN) | Falsos positivos (FP) |
| **Real Malignant** | Falsos negativos (FN) | Verdaderos positivos (TP) |

- **Diagonal principal (TN + TP)** → casos correctamente clasificados.  
- **Fuera de la diagonal (FP + FN)** → errores del modelo.  

> 💡 En diagnóstico médico, es especialmente importante **minimizar los falsos negativos (FN)**,  
> ya que representan casos malignos que el modelo no detectó correctamente.

---

### 💬 En resumen

> Esta evaluación muestra el **rendimiento real del modelo sobre imágenes nuevas**, aplicando el umbral óptimo encontrado.  
> Un buen modelo debe mostrar:
> - Alta *precision* y *recall* en la clase *Malignant*.
> - Un *F1-score* equilibrado entre ambas clases.
> - Una matriz de confusión con la mayoría de los valores concentrados en la diagonal principal.


In [ ]:
# === Evaluación final con umbral óptimo ===
from sklearn.metrics import classification_report, confusion_matrix

test_probs, test_y = collect_probs_labels(model, test_dl, malignant_idx=1)
test_pred = (test_probs >= OPT_THRESH).astype(int)

print("\n📋 Reporte con umbral óptimo:")
print(classification_report(test_y, test_pred, target_names=test_ds.classes))
print("Matriz de confusión (umbral ajustado):\n", confusion_matrix(test_y, test_pred))


## 🎨 Interpretación de la matriz de confusión ajustada

Esta celda **no calcula nuevas métricas**, sino que **representa visualmente** la matriz de confusión obtenida en la evaluación final.

El objetivo es facilitar la **interpretación visual de los aciertos y errores** del modelo:

- El **color más oscuro** en la diagonal indica una mayor cantidad de aciertos.  
- Las **celdas fuera de la diagonal** representan errores:
  - Falsos positivos (el modelo predijo *Malignant* pero era *Benign*).
  - Falsos negativos (el modelo predijo *Benign* pero era *Malignant*).

> 💡 En diagnóstico médico, es fundamental que la celda correspondiente a **Malignant → Malignant** (parte inferior derecha) tenga un color intenso,  
> ya que eso indica una **alta detección de casos realmente malignos**.

Además, esta versión:
- Usa un **mapa de color azul ("Blues")** para mostrar las intensidades.
- Añade **etiquetas numéricas** dentro de cada celda.
- Guarda la imagen como `confusion_matrix_adjusted.png` para incluirla en informes o presentaciones.

> En resumen, esta celda no cambia los resultados, pero **mejora la comprensión visual** de los aciertos y errores del modelo en el conjunto de test.


In [ ]:
# === Matriz de confusión AJUSTADA (grande y con colores) ===
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# --- Usa la versión compatible de collect_probs_labels (la que admite f) ---
test_probs, test_y = collect_probs_labels(model, test_dl, malignant_idx=1)  # p(Malignant)
test_pred = (test_probs >= OPT_THRESH).astype(int)

# --- Matriz de confusión ---
cm = confusion_matrix(test_y, test_pred)

# --- Plot grande y con colorbar ---
fig, ax = plt.subplots(figsize=(8.5, 7.5))
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=test_ds.classes)
disp.plot(cmap="Blues", values_format='d', ax=ax, colorbar=True)

ax.set_title(f"Matriz de Confusión (Test) — Umbral óptimo = {OPT_THRESH:.2f}",
             fontsize=16, pad=12)
ax.set_xlabel("Predicción", fontsize=13)
ax.set_ylabel("Etiqueta real", fontsize=13)
plt.xticks(fontsize=11)
plt.yticks(fontsize=11)
plt.tight_layout()
plt.show()

# --- (opcional) Guardar a disco ---
fig, ax = plt.subplots(figsize=(8.5, 7.5))
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=test_ds.classes)
disp.plot(cmap="Blues", values_format='d', ax=ax, colorbar=True)
ax.set_title(f"Matriz de Confusión (Test) — Umbral óptimo = {OPT_THRESH:.2f}")
plt.tight_layout()
plt.savefig("confusion_matrix_adjusted.png", dpi=220, bbox_inches="tight")
plt.close()
print("✅ Guardada en: confusion_matrix_adjusted.png")


## 📊 Celda 7.5 – Comparativa y análisis de generalización

Muestra un resumen global de métricas (Loss, Accuracy, F1, Recall, Precision) para *train*, *val* y *test*.  
También calcula los **gaps de generalización** entre entrenamiento y test.

> 💬 Interpretación:
> - Si el *gap* entre train y test es **< 2% → excelente generalización**.  
> - Entre 2–5% → buena generalización.  
> - > 5% → *overfitting*.  
> - Gap negativo grande → *underfitting*.  
>
> El gráfico permite ver si el modelo mantiene un rendimiento estable fuera del conjunto de entrenamiento.


In [ ]:
# === Celda 7.5: Evaluación global y generalización del modelo ===
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score, recall_score, precision_score

# --- Métricas finales de entrenamiento ---
final_train_loss = history["train_loss"][-1]
final_val_loss   = history["val_loss"][-1]
final_train_acc  = history["train_acc"][-1]
final_val_acc    = history["val_acc"][-1]

# --- Métricas de test (usando OPT_THRESH) ---
test_probs, test_y = collect_probs_labels(model, test_dl, malignant_idx=1)
test_pred = (test_probs >= OPT_THRESH).astype(int)

test_acc = np.mean(test_pred == test_y)
test_f1  = f1_score(test_y, test_pred, pos_label=1)
test_rec = recall_score(test_y, test_pred, pos_label=1)
test_pre = precision_score(test_y, test_pred, pos_label=1)

# --- Gaps de generalización (diferencias porcentuales) ---
gap_train_val  = 100 * (final_train_acc - final_val_acc)
gap_train_test = 100 * (final_train_acc - test_acc)

# --- Tabla resumen ---
metrics_df = pd.DataFrame({
    "Conjunto": ["Entrenamiento", "Validación", "Test"],
    "Loss": [final_train_loss, final_val_loss, np.nan],
    "Accuracy": [final_train_acc, final_val_acc, test_acc],
    "F1": [np.nan, np.nan, test_f1],
    "Recall": [np.nan, np.nan, test_rec],
    "Precision": [np.nan, np.nan, test_pre]
})

display(metrics_df.round(4))

# --- Gráfico de barras comparativo ---
plt.style.use("seaborn-v0_8-muted")
fig, ax = plt.subplots(figsize=(8, 5))
sets = ["Entrenamiento", "Validación", "Test"]
accs = [final_train_acc, final_val_acc, test_acc]
bars = ax.bar(sets, accs, color=["#57c057", "#c662c1", "#1f77b4"], alpha=0.85)

ax.set_ylim(0, 1)
ax.set_ylabel("Accuracy", fontsize=12)
ax.set_title("Comparativa de precisión final", fontsize=14)
ax.bar_label(bars, fmt="%.3f", padding=3)
ax.grid(axis="y", linestyle="--", alpha=0.6)
plt.tight_layout()
plt.show()

# --- Análisis de generalización ---
print("📊 Evaluación de la generalización del modelo:\n")
print(f"▪ Diferencia Entrenamiento–Validación: {gap_train_val:+.2f}%")
print(f"▪ Diferencia Entrenamiento–Test:       {gap_train_test:+.2f}%")

if abs(gap_train_test) < 2:
    msg = "Excelente generalización 💎 (modelo estable y sin overfitting)."
elif abs(gap_train_test) < 5:
    msg = "Buena generalización 👍 (ligero gap esperable en problemas reales)."
elif gap_train_test > 5:
    msg = "⚠️ Posible overfitting: el modelo rinde mejor en train que en test."
else:
    msg = "⚠️ Posible underfitting: el modelo no llega a aprender bien los patrones."

print(f"\n🔎 Diagnóstico: {msg}")


## 🧠 Interpretación de la gráfica de generalización

La gráfica compara la **precisión (accuracy)** del modelo en los tres conjuntos de datos:

- 🟩 **Entrenamiento (Train)** → mide cómo de bien el modelo aprende los patrones conocidos.  
- 🟪 **Validación (Val)** → evalúa el rendimiento en datos no usados durante el aprendizaje (control del *overfitting*).  
- 🟦 **Test** → mide la capacidad del modelo para generalizar a datos completamente nuevos.

---

### 📊 Cómo interpretar las diferencias

| Métrica | Significado | Interpretación |
|:--------|:-------------|:---------------|
| **Train alto y Val/Test similares** | El modelo aprende bien sin memorizar. | ✅ Buen equilibrio y generalización. |
| **Train ≫ Val/Test (gap > 5%)** | El modelo memoriza los datos de entrenamiento. | ⚠️ *Overfitting* — mala generalización. |
| **Train ≈ Val ≈ Test pero todos bajos** | El modelo no aprende patrones relevantes. | ⚠️ *Underfitting* — poca capacidad o datos insuficientes. |
| **Val o Test ligeramente menor (2–5%)** | Diferencia esperable en la práctica. | 👍 Modelo estable y bien regularizado. |

---

### 📈 Qué observar en la gráfica

- Si las barras de **Validación** y **Test** están **cercanas entre sí**, el modelo **generaliza correctamente**.  
- Si la barra de **Entrenamiento** está muy por encima, el modelo **sobreajusta**.  
- Si todas las barras están bajas, el modelo **no está aprendiendo** los patrones de forma efectiva.  

---

### 🧮 Gap de generalización

La diferencia porcentual entre *Train* y *Test* indica el nivel de generalización:

| Gap (`train - test`) | Diagnóstico |
|:---------------------:|:-------------|
| **< 2 %** | 💎 Excelente generalización |
| **2–5 %** | 👍 Buena generalización |
| **> 5 %** | ⚠️ Posible *overfitting* |
| **Gap negativo grande** | ⚠️ *Underfitting* — el modelo rinde peor incluso en entrenamiento |

---

### 💬 En resumen

> Un modelo **bien generalizado** mantiene un rendimiento similar entre *train*, *val* y *test*,  
> lo que indica que ha aprendido **patrones reales** y no solo ha memorizado las imágenes de entrenamiento.


In [ ]:
# === Celda 8: Curvas de pérdida y precisión (versión final) ===
import matplotlib.pyplot as plt

plt.style.use('seaborn-v0_8-muted')

epochs_range = range(1, len(history["train_loss"]) + 1)
fig, ax = plt.subplots(1, 2, figsize=(13, 5))

# --- Pérdida ---
ax[0].plot(epochs_range, history["train_loss"], "o-", color="#1f77b4", label="Entrenamiento")
ax[0].plot(epochs_range, history["val_loss"], "o--", color="#ff7f0e", label="Validación")
ax[0].set_title("Evolución de la pérdida", fontsize=13)
ax[0].set_xlabel("Época"); ax[0].set_ylabel("Loss")
ax[0].legend(frameon=False); ax[0].grid(ls="--", alpha=0.6)

# --- Precisión ---
ax[1].plot(epochs_range, history["train_acc"], "o-", color="#57c057", label="Entrenamiento")
ax[1].plot(epochs_range, history["val_acc"], "o--", color="#c662c1", label="Validación")
ax[1].set_title("Evolución de la precisión", fontsize=13)
ax[1].set_xlabel("Época"); ax[1].set_ylabel("Accuracy")
ax[1].legend(frameon=False); ax[1].grid(ls="--", alpha=0.6)

plt.suptitle("Curvas de entrenamiento del modelo", fontsize=15, y=1.02)
plt.tight_layout()
plt.savefig("curvas_entrenamiento.png", dpi=220, bbox_inches="tight")
plt.show()
print("✅ Guardadas como: curvas_entrenamiento.png")
